### Installation

In [ ]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.6.

### Unsloth

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "Qwen/Qwen3-0.6B",
    max_seq_length = 1024,
    load_in_4bit = False,
    load_in_8bit = False,
    full_finetuning = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-3",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files='aij_task_1_train.csv')['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset[100]

{'id': '16c45b04275a3a2db041120ed378c530',
 'prompt': '### Задание для оценки:\nСложите два числа:\n7309 + 82783 = \n\n### Эталонный ответ:\n90092\n\n### Ответ для оценки:\n 83512\n\n### Критерий оценки:\nПравильность ответа и формата\n\n### Шкала оценивания по критерию:\n0: Ответ содержит ошибки и не выполнены требования из инструкции.\n\n1: Формат ответа совпадает с требуемым в инструкции, но ответ содержит ошибки.\n\n2: Ответ правильный, но не выполнены требования из инструкции.\n\n3: Ответ правильный и соответствует требованиям формата.\n',
 'score': 1}

In [ ]:
def formatting_prompts_func(examples):
    messages = [[{'role': 'user', 'content': prompt}, {'role': 'assistant', 'content': str(int(score))}] for prompt, score in zip(examples['prompt'], examples['score'])]
    texts = [tokenizer.apply_chat_template(message, tokenize = False, add_generation_prompt = False) for message in messages]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/5197 [00:00<?, ? examples/s]

In [ ]:
dataset[100]["text"]

'<|im_start|>user\n### Задание для оценки:\nСложите два числа:\n7309 + 82783 = \n\n### Эталонный ответ:\n90092\n\n### Ответ для оценки:\n 83512\n\n### Критерий оценки:\nПравильность ответа и формата\n\n### Шкала оценивания по критерию:\n0: Ответ содержит ошибки и не выполнены требования из инструкции.\n\n1: Формат ответа совпадает с требуемым в инструкции, но ответ содержит ошибки.\n\n2: Ответ правильный, но не выполнены требования из инструкции.\n\n3: Ответ правильный и соответствует требованиям формата.\n<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1<|im_end|>\n'

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

### Train the model

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_ratio = 0.05,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        logging_steps = 5,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        dataset_num_proc=2,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4157 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n<think>\n\n</think>\n\n",
)

Map (num_proc=2):   0%|          | 0/4157 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|im_start|>user\n### Задание для оценки:\nРеши простую математическую задачу. В качестве ответы выведи только одно число. Задача:\n618 + 631 = \nОтвет:\n\n### Эталонный ответ:\n1249\n\n### Ответ для оценки:\n618 + 631 = 1249.\n\n### Критерий оценки:\nПравильность ответа и формата\n\n### Шкала оценивания по критерию:\n0: Ответ содержит ошибки и не выполнены требования из инструкции.\n\n1: Формат ответа совпадает с требуемым в инструкции, но ответ содержит ошибки.\n\n2: Ответ правильный, но не выполнены требования из инструкции.\n\n3: Ответ правильный и соответствует требованиям формата.\n<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n2<|im_end|>\n'

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                              2<|im_end|>\n'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.326 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,157 | Num Epochs = 1 | Total steps = 1,040
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 596,049,920/596,049,920 (100.00% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,9.117000
10,6.784600
15,2.515700
20,0.548500
25,0.507400
30,0.386100
35,0.519200
40,0.457100
45,0.513000
50,0.495200


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

949.8303 seconds used for training.
15.83 minutes used for training.
Peak reserved memory = 11.807 GB.
Peak reserved memory for training = 9.481 GB.
Peak reserved memory % of max memory = 80.096 %.
Peak reserved memory for training % of max memory = 64.317 %.


<a name="Inference"></a>
### Inference

In [ ]:
messages = [{
    "role": "user",
    "content": dataset['test']['prompt'][0]
}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    enable_thinking=False,
    add_generation_prompt = True,
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 10,
)
tokenizer.batch_decode(outputs)

['<|im_start|>user\n### Задание для оценки:\nВыполните арифметическую операцию. В качестве ответа напишите число, которое получается после ее выполнения.\n28262 + 33424 = \n\n### Эталонный ответ:\n61686\n\n### Ответ для оценки:\n51786\n\n### Критерий оценки:\nПравильность ответа и формата\n\n### Шкала оценивания по критерию:\n0: Ответ содержит ошибки и не выполнены требования из инструкции.\n\n1: Формат ответа совпадает с требуемым в инструкции, но ответ содержит ошибки.\n\n2: Ответ правильный, но не выполнены требования из инструкции.\n\n3: Ответ правильный и соответствует требованиям формата.\n<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1<|im_end|>']

<a name="Save"></a>
### Saving

In [ ]:
model.save_pretrained("aij_qwen_0.6b")
tokenizer.save_pretrained("aij_qwen_0.6b")

('aij_qwen_0.6b/tokenizer_config.json',
 'aij_qwen_0.6b/special_tokens_map.json',
 'aij_qwen_0.6b/chat_template.jinja',
 'aij_qwen_0.6b/vocab.json',
 'aij_qwen_0.6b/merges.txt',
 'aij_qwen_0.6b/added_tokens.json',
 'aij_qwen_0.6b/tokenizer.json')